In [84]:
import pandas as pd

In [85]:
filename = '/content/drive/Othercomputers/My Laptop/sales_predictions/sales_predictions.csv'

In [86]:
#Import a csv file
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [88]:
#Replace missing values in Item Weight column with the mean of the population
df.loc[df['Item_Weight'].isna(), 'Item_Weight'] = df.loc[df['Item_Weight'].isna(), 'Item_Weight'].fillna(df['Item_Weight'].mean())

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [90]:
df['Item_Weight'].value_counts()

12.857645    1463
12.150000      86
17.600000      82
13.650000      77
11.800000      76
             ... 
7.560000        2
9.420000        1
5.400000        1
6.520000        1
7.685000        1
Name: Item_Weight, Length: 416, dtype: int64

In [91]:
#Content conscious customers
#might purchase based on this description, plus field can be converted into 0 
#for regular and 1 for Low Fat. Arriving at two values for this column is a 
#good thing
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [92]:
#Performing the replace on the improper category descriptions
df.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'}},inplace=True)

In [93]:
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [94]:
#Checking the Item Type for any necessary category item cleanup
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [95]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Size'],dropna=False,as_index=False)[['Item_Identifier']].count()

,Outlet_Location_Type,Outlet_Type,Outlet_Size,Item_Identifier
0,Tier 1,Grocery Store,Small,528
1,Tier 1,Supermarket Type1,Medium,930
2,Tier 1,Supermarket Type1,Small,930
3,Tier 2,Supermarket Type1,Small,930
4,Tier 2,Supermarket Type1,NaN,1855
5,Tier 3,Grocery Store,NaN,555
6,Tier 3,Supermarket Type1,High,932
7,Tier 3,Supermarket Type2,Medium,928
8,Tier 3,Supermarket Type3,Medium,935


In [96]:
df[df['Outlet_Size'].isna()].describe(include='all')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
count,2410,2410.000000,2410,2410.000000,2410,2410.000000,2410,2410.000000,0,2410,2410,2410.000000
unique,1393,NaN,2,NaN,16,NaN,3,NaN,0,2,2,NaN
top,FDB28,NaN,Low Fat,NaN,Fruits and Vegetables,NaN,OUT045,NaN,NaN,Tier 2,Supermarket Type1,NaN
freq,3,NaN,1562,NaN,349,NaN,929,NaN,NaN,1855,1855,NaN
mean,NaN,12.778479,NaN,0.070259,NaN,140.322946,NaN,2003.000000,NaN,NaN,NaN,1822.626947
std,NaN,4.589825,NaN,0.055029,NaN,62.397904,NaN,3.506502,NaN,NaN,NaN,1557.858015
min,NaN,4.590000,NaN,0.000000,NaN,32.090000,NaN,1998.000000,NaN,NaN,NaN,33.290000
25%,NaN,8.785000,NaN,0.028744,NaN,93.096200,NaN,2002.000000,NaN,NaN,NaN,554.777850
50%,NaN,12.600000,NaN,0.056414,NaN,142.165400,NaN,2002.000000,NaN,NaN,NaN,1443.454400
75%,NaN,16.700000,NaN,0.100364,NaN,185.135800,NaN,2007.000000,NaN,NaN,NaN,2681.509500


#My best judgement is to set the missing values for the Outlet Type to small based on the analyis using GroupBy and Describe methods

In [97]:
df.loc[df['Outlet_Size'].isna(), 'Outlet_Size'] = df.loc[df['Outlet_Size'].isna(), 'Outlet_Size'].fillna("Small")

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


#First Checkpoint
##Checkpoint is provided just before performing removal of any duplicate rows

In [99]:
df1_copy = df

In [100]:
df.shape

(8523, 12)

In [101]:
df.drop_duplicates(inplace=True)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB


In [103]:
df.shape

(8523, 12)

#Appears there are no duplicates in the dataframe; record counts did not change after the drop duplicates step